In [ ]:
!pip install torch transformers

In [ ]:
from transformers import pipeline

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('my_recipes_data.csv')

In [ ]:
recipes = df['recipe'].tolist()

In [ ]:
import re

In [ ]:
import random

Есть 500 начальных предложений. Генерируем к ним продолжение. При этом записываем в словарь еще и настоящие предложения из датафрейма, следующие за исходными.

In [ ]:
recipes = [re.sub(r'\d+\.', '', str(text)) for text in recipes]
recipes = [text.replace('\n',' ').replace('\t',' ') for text in recipes]
recipes = random.sample(recipes, 1000)

In [ ]:
import regex

In [ ]:
recipes = [regex.split(r'(?<=[.!?])\s+(?=\p{Lu})', text) for text in recipes]
recipes

In [ ]:
data = dict()

In [ ]:
for recipe in recipes:
    if len(list(data.keys())) < 500:
        if len(recipe) > 2:
            index = random.randint(0, len(recipe)-2)
            data[recipe[index]] = [recipe[index+1], None]
data

In [ ]:
len(data.keys())

500

In [ ]:
from tqdm import tqdm

In [ ]:
initial_prompts = list(data.keys())
initial_prompts

In [ ]:
generator = pipeline('text-generation', model="sberbank-ai/rugpt3large_based_on_gpt2" )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
for p in tqdm(range(100)):
    generated_text = generator(initial_prompts[p], max_new_tokens=25, num_return_sequences=1)
    data[initial_prompts[p]][1] = generated_text[0]['generated_text']

100%|██████████| 100/100 [15:13<00:00,  9.13s/it]


In [ ]:
for p in tqdm(range(100, 200)):
    generated_text = generator(initial_prompts[p], max_new_tokens=25, num_return_sequences=1)
    data[initial_prompts[p]][1] = generated_text[0]['generated_text']

100%|██████████| 100/100 [14:36<00:00,  8.77s/it]


In [ ]:
for p in tqdm(range(201, 300)):
    generated_text = generator(initial_prompts[p], max_new_tokens=25, num_return_sequences=1)
    data[initial_prompts[p]][1] = generated_text[0]['generated_text']

100%|██████████| 99/99 [14:45<00:00,  8.95s/it]


In [ ]:
for p in tqdm(range(301, 400)):
    generated_text = generator(initial_prompts[p], max_new_tokens=25, num_return_sequences=1)
    data[initial_prompts[p]][1] = generated_text[0]['generated_text']

100%|██████████| 99/99 [14:22<00:00,  8.72s/it]


In [ ]:
generated_text = generator(initial_prompts[300], max_new_tokens=25, num_return_sequences=1)
data[initial_prompts[300]][1] = generated_text[0]['generated_text']

In [ ]:
generated_text = generator(initial_prompts[400], max_new_tokens=25, num_return_sequences=1)
data[initial_prompts[400]][1] = generated_text[0]['generated_text']

In [ ]:
for p in tqdm(range(401, len(initial_prompts))):
    generated_text = generator(initial_prompts[p], max_new_tokens=25, num_return_sequences=1)
    data[initial_prompts[p]][1] = generated_text[0]['generated_text']

100%|██████████| 99/99 [15:37<00:00,  9.47s/it]


In [ ]:
generated_text = generator(initial_prompts[200], max_new_tokens=25, num_return_sequences=1)
data[initial_prompts[200]][1] = generated_text[0]['generated_text']

Получаем датафрейм, где первый столбец -- это исконная фраза, второй -- следующая за ним фраза, третий -- сгенерированный на основании исконной фразы текст.

In [ ]:
df_new = pd.DataFrame(list(data.items()), columns=['initial sentence', 'value'])
df_new[['true sentence', 'generated text']] = pd.DataFrame(df_new['value'].tolist())
del df_new['value']
df_new

,initial sentence,true sentence,generated text
0,Берем любую глубокую посуду и начинаем выклады...,Так чтобы самым последним оказался сахарный слой.,Берем любую глубокую посуду и начинаем выклады...
1,"В глубокой миске соедините два вида фарша, пе...",Сформируйте из фарша шарики среднего размера.,"В глубокой миске соедините два вида фарша, пе..."
2,Чуть отжимаем руками.,"Теперь поместим огурец в соусницу, добавим спе...",Чуть отжимаем руками.\n\nНачинаем сгибать руки...
3,Обжариваем ребрышки до золотистого цвета на ра...,Присыпаем готовое блюдо кунжутом.,Обжариваем ребрышки до золотистого цвета на ра...
4,"Я обычно ставлю недалеко от плиты, поэтому пол...",Готовую капусту уберите в прохладное место для...,"Я обычно ставлю недалеко от плиты, поэтому пол..."
...,...,...,...
495,"Чтобы всё получилось без сюрпризов, рекомендую...",Или же смажьте растительным маслом.,"Чтобы всё получилось без сюрпризов, рекомендую..."
496,Добавьте нарезанные огурцы.,Нарежьте зелень.,Добавьте нарезанные огурцы.\n\nСалат из огурцо...
497,На каждой половинке у кочерыжки сделайте небол...,Тщательно промойте каждый лист пекинской капус...,На каждой половинке у кочерыжки сделайте небол...
498,"А затем подаем мясо вместе с овощами, полив ос...",Вот и все!,"А затем подаем мясо вместе с овощами, полив ос..."


In [ ]:
df_new.to_csv('generated_texts.csv')

Для того, чтобы тексты из второго и третьего столбца меньше отличались по формату, произведем форматирование.

In [ ]:
modified = df_new['generated text'].tolist()

In [ ]:
for i in range(len(modified)):
    modified[i] = re.sub(r'([.,!?])([А-ЯЁ])', r'\1 \2',
                         modified[i].replace(initial_prompts[i], '').replace('\n','').replace('\xa0', ' ').strip())
    if modified[i][-1].isdigit() or modified[i][-1].isalpha():
        modified[i] += '...'
    new_phrase = ""
    for j in range(len(modified[i])):
        if modified[i][j].isupper() and j > 0 and modified[i][j-1] != ' ':
            new_phrase += '. '
        new_phrase += modified[i][j]

    for j in range(len(modified[i])):
        if modified[i][j].isdigit() and j > 0 and modified[i][j-1].isalpha():
            new_phrase += ', '
        new_phrase += modified[i][j]

    modified[i] = new_phrase

modified

In [ ]:
df_new['generated text'] = modified

In [ ]:
true_modified = df_new['true sentence'].tolist()

In [ ]:
for t in range(len(true_modified)):
    c = random.randint(10, 25)
    if len(true_modified[t].split(' ')) > c:
        true_modified[t] = ' '.join(true_modified[t].split(' ')[:c]) + '...'

true_modified

In [ ]:
df_new['true sentence'] = true_modified

In [ ]:
df_new.to_csv('generated_texts.csv')

Продолжаем форматирование. Удаляем спецсимволы.

In [ ]:
df = pd.read_csv('generated_texts.csv')
df

In [183]:
def remove_special_chars(cell):
    return cell.replace('\n', '').replace('\t', '').replace('\r', '')

df_cleaned = df[['initial sentence', 'true sentence', 'generated text']].applymap(remove_special_chars)

In [184]:
import re

In [185]:
d = df_cleaned['generated text'].tolist()

In [187]:
for text in range(len(d)):
    text_list = d[text].split('.')
    unique_text_parts = []
    for part in text_list:
        if part.strip() not in unique_text_parts:
            unique_text_parts.append(part.strip())
    d[text] = '. '.join(unique_text_parts)

In [188]:
for t in range(len(d)):
    if '. . ' in d[t]:
        d[t] = d[t].split('. . ')[0]

In [189]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ',' and d[t][i+1].isupper():
            d[t] = d[t].split(f'{d[t][i]}{d[t][i+1]}')[0]
            break

In [191]:
for t in range(len(d)):
    d[t] = d[t].replace('• ', '').replace(' •', '')

In [192]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ';' and d[t][i+1].isupper():
            d[t] = d[t].split(f'{d[t][i]}{d[t][i+1]}')[0]
            break

In [193]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ';' and d[t][i+2].isupper():
            d[t] = d[t].split(f'{d[t][i]}{d[t][i+1]}')[0]
            break

In [194]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ':' and d[t][i+2].isupper():
            d[t] = d[t].split(f'{d[t][i]}{d[t][i+1]}')[0]
            break

In [195]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ':' and d[t][i+1] != ' ':
            d[t] = d[t].replace(':', ': ')

In [196]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i].isalpha() and d[t][i+1].isdigit():
            d[t] = d[t].replace(f'{d[t][i]}{d[t][i+1]}', f'{d[t][i]} {d[t][i+1]}')


In [197]:
for t in range(len(d)):
    for i in range(len(d[t])-2):
        if d[t][i] == ';' and d[t][i+1] != ' ':
            d[t] = d[t].replace(';', '; ')

In [198]:
for t in range(len(d)):
    d[t] = d[t].replace('Ингредиенты ', 'Ингредиенты: ')

In [199]:
for t in range(len(d)):
    if ' . ' in d[t]:
        d[t] = d[t].replace(' . ', '')

In [200]:
for t in range(len(d)):
    d[t] = d[t].rstrip().rstrip(',').rstrip('—')

In [201]:
for t in range(len(d)):
    if d[t][-1] == '.':
        d[t] = d[t]+'..'
    else:
        d[t] = d[t]+'...'

In [ ]:
d

In [203]:
df_cleaned['generated text'] = d

In [206]:
f = df_cleaned['true sentence'].tolist()
for i in range(len(f)):
    if '...' in f[i]:
        pass
    elif f[i][-1] == '.':
        f[i] = f[i]+'..'
    else:
        f[i] = f[i]+'...'
    f[i] = f[i].replace('......', '...')

In [208]:
df_cleaned['true sentence'] = f

In [209]:
df_cleaned

,initial sentence,true sentence,generated text
0,Берем любую глубокую посуду и начинаем выклады...,Так чтобы самым последним оказался сахарный сл...,"Затем добавляем корицу, гвоздику, ваниль, кори..."
1,"В глубокой миске соедините два вида фарша, пе...",Сформируйте из фарша шарики среднего размера...,"На дно формы положите половину теста, сверху п..."
2,Чуть отжимаем руками.,"Теперь поместим огурец в соусницу, добавим спе...",Начинаем сгибать руки в локтях. Сгибаем руки в...
3,Обжариваем ребрышки до золотистого цвета на ра...,Присыпаем готовое блюдо кунжутом...,"Добавляем к ним лук, морковь, помидоры, томатн..."
4,"Я обычно ставлю недалеко от плиты, поэтому пол...",Готовую капусту уберите в прохладное место для...,Как приготовить вкусный салат из капусты? Сала...
...,...,...,...
495,"Чтобы всё получилось без сюрпризов, рекомендую...",Или же смажьте растительным маслом...,Для этого нужно взять лист бумаги и вырезать и...
496,Добавьте нарезанные огурцы.,Нарежьте зелень...,Салат из огурцов с помидорами. Ингредиенты. Ог...
497,На каждой половинке у кочерыжки сделайте небол...,Тщательно промойте каждый лист пекинской капус...,"Вставьте кочерыжку в отверстие, которое вы сде..."
498,"А затем подаем мясо вместе с овощами, полив ос...",Вот и все!...,Салат из курицы с овощами. Ингредиенты: 1 кури...


In [210]:
df_cleaned.to_csv('generated_texts.csv')